# Pandas II

In this notebook, we will build on what we've learned so far with pandas. We will also introduce the **requests** library which will allow us to grab data from a webpage.

## Reading Data from a Webpage

We can also use pandas to grab tabular data from webpages. To do this, we first have to import the **requests** library so we can make an http request to grab a webpage.

Before we make the web request, navigate to the webpage by clicking on the link below so that you can see what the webpage looks like.

https://worldpopulationreview.com/country-rankings/coffee-producing-countries

You should see a page that includes a table of countries with their coffee production.

Before we make the http request, we need to set the http header to include information about where the request is coming from. Websites may otherwise block requests.



In [20]:
# Import the requests library for making http requests
import requests as r
# Create a header that says the request is coming form a browser-like agent (this is to prevent the website blocking our request)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
}

In [21]:
# Make an http request to get the webpage with the following url
url = "https://worldpopulationreview.com/country-rankings/coffee-producing-countries"
page = r.get(url, headers = headers)

In [22]:
# Check that the request was successful. If so, the status code should be 200.
page.status_code

200

In [23]:
# Print the first few hundred characters of the webpage
# The actual content of the page, which is HTML, is stored in the 'content' attribute of the page object
print(page.content[0:200])

b'<!DOCTYPE html><html lang="en"> <head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1"><title>Coffee Producing Countries 2025</title><meta name="description" c'


Now we will use the **pandas.read_html()** method to grab the tables from the webpage and put them in a list called tables. The list will contain all the tables in the webpage.

In [24]:
# First import the pandas library
import pandas as pd
# read_html() will find all the tables in the webpage and put them in a list.
tables = pd.read_html(page.content)
# In this case, there is only one table, so the length of the list should be 1.
len(tables)

1

Each table in the HTML page is converted into a dataframe in the tables list. As there is only one table, let's store it in a separate variable called cd (for coffee data).

In [25]:
# Store the one table as a dataframe in the variable cd
cd = tables[0]
# Check the type of this variable. It should say 'pandas.core.frame.DataFrame'
type(cd)

pandas.core.frame.DataFrame

In [26]:
# Let's look at the table
cd

,Unnamed: 0,Country,Coffee Production 2022 (t)↓,Coffee Yield 2022
0,NaN,Brazil,3.2M,1694.3
1,NaN,Vietnam,2M,2979.0
2,NaN,Indonesia,794.8K,618.1
3,NaN,Colombia,665K,789.4
4,NaN,Ethiopia,496.2K,668.9
...,...,...,...,...
74,NaN,Sao Tome and Principe,8,58.5
75,NaN,Suriname,6,20.4
76,NaN,New Caledonia,2,137.5
77,NaN,Cook Islands,0,587.3


## Data Wrangling

We are now going to do some data 'wrangling' on the dataframe. Basically, we are going to fix some issues with the data as it present to make it easier to analyze.

You may notice there are some issues with the data in the table.

1. There is a column full of 'NaN' which we can delete.
2. Some of the column titles contain special characters which we don't need. To fix this we can rename the columns.
3. The coffee production column has a mixture of numbers and letters: 3.2M means 3.2 million (tonnes) and 794.8K mean 794.8 thousand (tonnes). Also these 'values' are currently stored as strings. We need to replace these with consistent numerical values.

Let's start by renaming the columns.

In [27]:
# Rename the columns
cd.columns = ['Not used', 'Country', '2022 Production (tonnes)', '2022 Yield (kg/hectare)']
cd

,Not used,Country,2022 Production (tonnes),2022 Yield (kg/hectare)
0,NaN,Brazil,3.2M,1694.3
1,NaN,Vietnam,2M,2979.0
2,NaN,Indonesia,794.8K,618.1
3,NaN,Colombia,665K,789.4
4,NaN,Ethiopia,496.2K,668.9
...,...,...,...,...
74,NaN,Sao Tome and Principe,8,58.5
75,NaN,Suriname,6,20.4
76,NaN,New Caledonia,2,137.5
77,NaN,Cook Islands,0,587.3


In [28]:
# Drop column 1
cd = cd.drop(columns=['Not used'])

In [29]:
# Check the dimensions of the dataframe. Should be 79 x 3 now that we've removed a column.
cd.shape

(79, 3)

Now we need to deal with the fact that the numbers in the '2022 Production (tonnes)' column are actula strings with different symbol to denote thousands and millions.

First, we will create two new columns. One called **Units** to indicate the units and one called **Prod (kt)** to store the numerical value of coffee production in thousands of tonnnes.

In [30]:
# Create a new column called 'Units' with the default value '-' (string)
cd['Units']="-"
# Create a new column called 'Prod (kt)' with the default value 0.0 (float)
cd['Prod (kt)']=0.0
cd

,Country,2022 Production (tonnes),2022 Yield (kg/hectare),Units,Prod (kt)
0,Brazil,3.2M,1694.3,-,0.0
1,Vietnam,2M,2979.0,-,0.0
2,Indonesia,794.8K,618.1,-,0.0
3,Colombia,665K,789.4,-,0.0
4,Ethiopia,496.2K,668.9,-,0.0
...,...,...,...,...,...
74,Sao Tome and Principe,8,58.5,-,0.0
75,Suriname,6,20.4,-,0.0
76,New Caledonia,2,137.5,-,0.0
77,Cook Islands,0,587.3,-,0.0


This next bit of code requires some explanation.

It goes through the '2022 Production (tonnes)' column row by row. If a row has an 'M' then the value in the 'Units' column is changed to 'M' and the numerical value is put in the last column. Similarly with 'K'.


In [31]:
num_rows = cd.shape[0]
for i in range(num_rows):
    if("M" in cd.iloc[i, 1]):
        cd.iloc[i, 3] = 'M'
        cd.iloc[i, 4] = float(cd.iloc[i, 1].replace("M" , ""))
    elif("K" in cd.iloc[i, 1]):
        cd.iloc[i, 3] ="K"
        cd.iloc[i, 4] = float(cd.iloc[i, 1].replace("K", ""))
    else:
        cd.iloc[i, 4] = float(cd.iloc[i, 1])

In [32]:
cd

,Country,2022 Production (tonnes),2022 Yield (kg/hectare),Units,Prod (kt)
0,Brazil,3.2M,1694.3,M,3.2
1,Vietnam,2M,2979.0,M,2.0
2,Indonesia,794.8K,618.1,K,794.8
3,Colombia,665K,789.4,K,665.0
4,Ethiopia,496.2K,668.9,K,496.2
...,...,...,...,...,...
74,Sao Tome and Principe,8,58.5,-,8.0
75,Suriname,6,20.4,-,6.0
76,New Caledonia,2,137.5,-,2.0
77,Cook Islands,0,587.3,-,0.0


The last modification we will make is to change the value in the last column to be the actual numerical value in thousands of tonnes.

This means multiplying anything in the

In [ ]:
# Now update numerical value based on units
for i in range(len(cd['Production'])):
    if(cd.iloc[i, 2]=="M"):
        cd.iloc[i, 3] *=1000
    elif(cd.iloc[i, 2]=="-"):
        cd.iloc[i, 3] /= 1000